Referencias (para equipo docente):
- bounding box chile continental: extraído manualmente de [boundingbox.klokantech.com](https://boundingbox.klokantech.com/) \[[[-79.6157454929,-56.72500008],[-66.07534742,-56.72500008],[-66.07534742,-17.49839982],[-79.6157454929,-17.49839982],[-79.6157454929,-56.72500008]]\]  
**lon**: \[-79.6157454929, -66.07534742\], **lat**\[-56.72500008, -17.49839982\]
- extender singledispatch a métodos de [stackoverflow](https://stackoverflow.com/questions/24601722/how-can-i-use-functools-singledispatch-with-instance-methods)

In [ ]:
%load_ext autoreload
%autoreload 2
import requests
import json
import numpy as np
from scipy import stats
from datetime import date, datetime
from time import sleep

# Ejercicio 1
**Objetivo**: Calcular estadísticas descriptivas sobre los movimientos telúricos ocurridos en Chile en los últimos 100 años, haciendo uso de algunas de las herramientas del curso.

En pos de dicho objetivo utilizaremos una [API gratuita del _United States Geological Survey_ (USGS)](https://earthquake.usgs.gov/fdsnws/event/1/). Una API - Aplication Programming Interface o interfaz de programación de aplicaciones - es un conjunto de rutinas, protocolos y herramientas para crear aplicaciones de software. Básicamente, una API especifica cómo deben interactuar los componentes de software. - [Por Vangie Beal en Webopedia](https://www.webopedia.com/TERM/A/API.html)

En este ejercicio trabajaremos con un tipo especial de API, que son las [web API](https://en.wikipedia.org/wiki/Web_API) y particularmente con las web API que están diseñadas para descargar datos de servidores en un forma programática.

En particular, utilizaremos la API de _USGS_ para recolectar la información dichos movimientos telúricos ocurridos entre 1920 y 2020. Los pasos a seguir serán los siguientes:
1. Construir las consultas de la API
2. Recolectar los datos interactuando con la API
3. Analizar los resultados mediante numpy y scipy


## Preliminares

Antes de comenzar, preste especial atención a al siguiente decorador. Este extiende el decorador `functools.singledispatch` para manejar métodos en vez de funciones. Normalmente, este decorador está disponible a partir de python 3.8 [link](https://docs.python.org/3/library/functools.html#functools.singledispatchmethod), pero como es posible observar a continuación, con la ayuda de 
las herramientas que esta librería ya proporciona en Python 3.7, es posible extender fácilmente la funcionalidad de `functools.singledispatch` como es deseado:

**OBS**: No se explica el esta función pues es parte del ejercicio comprender su funcionamiento. Esto será evaluado más abajo.

In [ ]:
from functools import singledispatch, update_wrapper


def singledispatchmeth(func):

    despachador = singledispatch(func)

    def wrapper(*args, **kw):
        return despachador.dispatch(args[1].__class__)(*args, **kw)
    
    wrapper.register = despachador.register
    wrapper.registry = despachador.registry
    update_wrapper(wrapper, func)
    return wrapper

**OBS**: Este decorador será utilizado en dos ocasiones en el ejercicio, por lo que vale la pena comprender su funcionamiento.

## Construcción de consultas

Para realizar una consulta a la API de USGS utilizatemos el protocolo http, donde definimos por medio de la URL, los parámetros de nuestra consulta:

    https://earthquake.usgs.gov/fdsnws/event/1/[[METODO][?PARAMETROS]] 
    
En el campo [METODO]() utilizaremos "application.json" lo que implica que la respuesta entregada por la API vendrá en formato `.json`, es decir _JavaScript Object Notation_. Explicar en detalle este formato escapa del alcance del ejercicio. Para efectos prácticos es un formato que permite almacenar información mediante una escritura analoga a los `dict` de Python, donde además se permiten objetos análogos tipos `list`, `int`, `float`, `str`, `bool` entre otros.

En cuanto al campo [PARAMETROS], según está explicado en la documentación de la [API](https://earthquake.usgs.gov/fdsnws/event/1/) para hacer una consulta podemos definir una diversidad de parámetros. Los que serán útiles para el ejercicio son:
- Geográficos: que servirán para definir la zona geográfica de interés. Estos son: `minlatitude`, `minlongitude`, `maxlatitude` y `maxlongitude`.
- Temporales: servirán para definir el periodo de consulta. Se trata de: `starttime` y `endtime`.

Dado que la API entrega un máximo de 20.000 respuestas por consulta y que Chile es un país con mucha actividad sísmica es necesario realizar múltiples consultas para capturar todos movimientos telúricos en cuestión. Las múltiples consultas **variarán su periodo de interés**, pero la **zona geográfica quedará fija**.

Ya que es requerido trabajar con variables temporales, se hace uso de la clase `date` de la librería `datetime`. 

A modo de ejemplo para denotar el primero de enero de 1920, el objeto se puede inicializar con una tupla de largo 3 que contiene sólo enteros que serán interpretados de la forma `(año, mes, día)`. Es decir:

In [ ]:
primera_fecha = date(1920, 1, 1)
print(f'La representacion del objeto `primera_fecha` es: {repr(primera_fecha)}')
print(f'El string asociado `primera_fecha` es: {primera_fecha}')

1. Extienda la clase `date` de la librería `datetime` a una nueva clase que denominaremos `DateConSuma`, en la cual el operador `+` se comporte de igual forma que la clase `date`, excepto al operar con una tupla de largo tres que contenga sólo enteros. En este caso, la clase debe interpretar la tupla de la forma `(años a sumar, meses a sumar, días a sumar)`.
    - Utilice el decorador definido arriba `singledispatchmeth`.
    - Recuerde hacer uso de control de flujo para asegurar que se está operando con una tupla de las caracrerísticas  especificadas.
    - Haga uso de la función `super` adecuadamente. 

In [ ]:
from datetime import date

# defina la clase
class DateConSuma(date):
    
    @singledispatchmeth
    def __add__(self, sumando):
        return super().__add__(sumando)
    
    @__add__.register(tuple)
    def _(self, sumando):
        assert len(sumando) == 3, 'La operación `+` esta solo definida para tuplas de largo 3'
        assert all([isinstance(x, int) for x in sumando])
        nueva_fecha = self.replace(year=self.year + sumando[0])
        nueva_fecha = nueva_fecha.replace(month=self.month + sumando[1])
        nueva_fecha = nueva_fecha.replace(day=self.day + sumando[2])
        return nueva_fecha

- Compruebe que el resultado es el deseado:

In [ ]:
fecha_con_suma = DateConSuma(1920, 1, 1) 
fecha_con_suma + (10, 0, 0)

Note además que las operaciones de comparación funcionan como lo esperado:

In [ ]:
f'El dia de hoy {date.today()} sucedio antes que {fecha_con_suma}? {fecha_con_suma >= date.today()}'

Ahora que ya se tienen herramientas para trabajar con fechas, crearemos un iterable que entregue tuplas con las fechas necesarias para realizar las consultas.

2. Defina la clase `IteratorFechas` que retorne objetos de la clase `DateConSuma` espaciados cada $n$ años comenzando en 1920-01-01 y finalizando en 2020-01-01. Para ello:
    - Defina los atributos públicos `desde_fecha` y `dia_presente` en el constructor, ambos de la clase `DateConSuma`. Además el constructor dberá recibir el parametro `n` que guardará en el atributo `espaciado` que 
    - Defina el método mágico `__iter__` donde símplemente retorne el objeto.
    - Defina el método mágico `__next__` donde retorne una tupla de largo 2 que retorne los valores espaciados por 5 años. Para entregar la fecha 5 años después utilice la operación `+` ya definida en `DateConSuma`.
    - El output de las 3 primeras y últimas iteraciones debería ser las siguientes tuplas del objeto `IteratorFechas(5)`:
        ```python
        ('DateConSuma(1920, 1, 1)', 'DateConSuma(1925, 1, 1)')
        ('DateConSuma(1925, 1, 1)', 'DateConSuma(1930, 1, 1)')
        ('DateConSuma(1930, 1, 1)', 'DateConSuma(1935, 1, 1)')
        [...]
        ('DateConSuma(2005, 1, 1)', 'DateConSuma(2010, 1, 1)')
        ('DateConSuma(2010, 1, 1)', 'DateConSuma(2015, 1, 1)')
        ('DateConSuma(2015, 1, 1)', 'DateConSuma(2020, 1, 1)')

        ```
      Una vez entregada la última tupla i.e. ('DateConSuma(2010, 1, 1)', 'DateConSuma(2020, 1, 1)') es necesario que `__next__` levante la excepción `StopIteration` la próxima vez que sea llamado.

_Hint_ : Puede ser útil recordar como funciona un **iterator**. 

    equipo docente: El siguiente también puede ser escrito como un generador.

In [ ]:
# defina la clase
class IteratorFechas():
    
    def __init__(self, n):
        self.desde_fecha = DateConSuma(1920, 1, 1)
        self.dia_presente = date.today()
        self.espaciado = n
    
    def __iter__(self):
        return self
    
    def __next__(self):
        hasta_fecha = self.desde_fecha + (self.espaciado, 0, 0)
        
        if hasta_fecha <= self.dia_presente:
            viejo_desde_fecha = self.desde_fecha
            self.desde_fecha = hasta_fecha
            return (viejo_desde_fecha, hasta_fecha)
        
        else:
            raise StopIteration()

- Compruebe que la clase `GeneradorFechas` funciona como es esperado.

In [ ]:
for desde_, hasta_ in IteratorFechas(5):
    print((repr(desde_), repr(hasta_)))

Como fue enunciado anteriormente la consulta a la API de USGS se hace mediante la sintaxis:

    https://earthquake.usgs.gov/fdsnws/event/1/[METODO[?PARAMETROS]] 
    
Mencionamos además que los parámetros de interés son:
- Geográficos: `minlatitude`, `minlongitude`, `maxlatitude` y `maxlongitude`.
- Temporales: `starttime` y `endtime`

Así, por ejemplo si se desea consultar por los eventos ocurridos entre el primero de enero de 1920 y el primero de enero de 1925 la consulta sería de la forma:

[https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=1920-01-01&endtime=1925-01-01&minlatitude=-56.72500008&minlongitude=-80&maxlatitude=-17.49839982&maxlongitude=-66.07534742](https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=1920-01-01&endtime=1925-01-01&minlatitude=-56.72500008&minlongitude=-80&maxlatitude=-17.49839982&maxlongitude=-66.07534742)

Note que los parámetros geográficos fijos a utilizar son:

```python
'minlatitude': -56.72500008, 
'minlongitude': -80,
'maxlatitude': -17.49839982,
'maxlongitude': -66.07534742,
``` 

3. Para la siguiente clase `ConstructorConsultasUSGS` defina el método `construye_consulta` que use los atributos `params_fijos_dict` y `url_base`, y además reciba los argumentos `desde_fecha` y `hasta_fecha` para retornar un string de consulta.  
A modo de ejemplo, se espera que método `ConstructorConsultasUSGS.construye_consulta(DateConSuma(1920, 1, 1), DateConSuma(1925, 1, 1))`
retorne el string 
```python
'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=1920-01-01&endtime=1925-01-01&minlatitude=-56.72500008&minlongitude=-80&maxlatitude=-17.49839982&maxlongitude=-66.07534742'
```

In [ ]:
class ConstructorConsultasUSGS():
    
        
    def __init__(self):

        # atributos fijos de consulta
        self.params_dict = {
            'minlatitude': -56.72500008, 
            'minlongitude': -80,
            'maxlatitude': -17.49839982,
            'maxlongitude': -66.07534742,
        }
        self.url_base = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson"

    def construye_consulta(self, desde_fecha, hasta_fecha):
        
        iter_dict = {
            'starttime': desde_fecha,
            'endtime': hasta_fecha
        }
        iter_dict = {**iter_dict, **self.params_dict}
        lista_parametros = ['&'+str(k)+'='+str(v) for k, v in iter_dict.items()]
        return self.url_base + ''.join(lista_parametros)



- Compruebe que el resultado es el deseado:

In [ ]:
ConstructorConsultasUSGS().construye_consulta(
    DateConSuma(1920, 1, 1), DateConSuma(1925, 1, 1))

4. Cree la clase `GeneradorConsulta` que herede de las clases `ConstructorConsultasUSGS` e `IteratorFechas` para logrr un iterador que entregue las consultas a la API.  
A modo de ejemplo, las 3 primeras y últimas iteraciones deben retornar:
```python
'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=1920-01-01&endtime=1925-01-01&minlatitude=-56.72500008&minlongitude=-80&maxlatitude=-17.49839982&maxlongitude=-66.07534742'
'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=1925-01-01&endtime=1930-01-01&minlatitude=-56.72500008&minlongitude=-80&maxlatitude=-17.49839982&maxlongitude=-66.07534742'
'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=1930-01-01&endtime=1935-01-01&minlatitude=-56.72500008&minlongitude=-80&maxlatitude=-17.49839982&maxlongitude=-66.07534742'
...
'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2005-01-01&endtime=2010-01-01&minlatitude=-56.72500008&minlongitude=-80&maxlatitude=-17.49839982&maxlongitude=-66.07534742'
'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2010-01-01&endtime=2015-01-01&minlatitude=-56.72500008&minlongitude=-80&maxlatitude=-17.49839982&maxlongitude=-66.07534742'
'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2015-01-01&endtime=2020-01-01&minlatitude=-56.72500008&minlongitude=-80&maxlatitude=-17.49839982&maxlongitude=-66.07534742'
```
Para ello haga _overridding_ de los metodos:
    - `__init__`: haga uso de la función `super` en dos ocasiones, recordando _MRO_.
    - `__next__`: haga uso de la función `super` en dos ocasiones.

In [ ]:
class GeneradorConsultaUSGS(IteratorFechas, ConstructorConsultasUSGS):
        
    def __init__(self, n):
        
        super().__init__(n)
        super(IteratorFechas, self).__init__()

    def __next__(self):
        
        desde_fecha, hasta_fecha = super().__next__()
        return super().construye_consulta(desde_fecha, hasta_fecha)



- Compruebe que el resultado es el deseado:

In [ ]:
for consulta_str in GeneradorConsultaUSGS(5):
    print(consulta_str)

## Recolección de datos

Ahora que ya construimos la clase que itera sobre las diferentes consultas que debes ser realizadas a la API de USGS, procederemos con la recolección de los datos.

Los datos seran almacendos en un diccionario con la siguiente estructura:
```python
{
    'id': [id_1, id_2, ...],
    'lugar': [lugar_1, lugar_2, ...],
    'timestamp': [timestamp_1, timestamp_2, ...],
    'magnitud': [magnitud_1, magnitud_2, ...],
    'longitud': [longitud_1, longitud_2, ...],
    'latitud': [latitud_1, latitud_2, ...]
}
```
es decir contiene en las llaves `str` con los nombres del atributos a recolectar y en los valores asociados, listas que contienen los atributos respectivos de cada observación recolectada.

Ya que trabajaremos este tipo de estructura, definiremos una clase que simplifique la concatenación de los resultados en cada iteración.

5. Extienda la clase `dict` definiendo la clase `DictDeListas` donde se defina el método `extiende_listas` que reciba como argumento otro objeto del tipo `DictDeListas` y concatene las listas asociadas asociadas a las mismas llaves en ambos argumentos.
    - Haga uso del método `items`.  
   
   _Hint_ : Ponga atención a la celda subsiguiente de código para aclara dudas sobre el funcionamiento del método.

In [ ]:
class DictDeListas(dict):
    
    def extiende_listas(self, dict_de_listas):
        
        # asegure que el argumento es del tipo DictDeListas
        assert isinstance(dict_de_listas, DictDeListas), ValueError(
            'El argumento `dict_de_listas` debe ser de la clase `DictDeListas`.')
        
        for k, l in dict_de_listas.items():
            
            self[k] += l
            

- Compruebe que el resultado es el siguiente:  

```python
{'id': ['iscgem908354',
  'iscgem908107',
  'iscgem908989',
  'iscgem908986',
  'iscgem908971',
  'iscgem908457']}

```

In [ ]:
primer_dict_de_listas = DictDeListas({
    'id': ['iscgem908354', 'iscgem908107','iscgem908989']
})
primer_dict_de_listas.extiende_listas(DictDeListas({
    'id': ['iscgem908986', 'iscgem908971', 'iscgem908457']
}))
primer_dict_de_listas

Ya que tenemos definido el tipo de dato que contendrá las observaciones recolectadas, ahora podemos proceder con la realización de las consultas. Para ellos utilizaremos la librería `requests` y en particular el método `get` que permite efectuar consultas mediante el método _GET_ del protocolo _HTTP_. Explicar ambos conceptos se escapa del alcance de este ejercicio. Sin embargo es necesario examinar la clase `request.Response`. Por ejemplo, al ejecutar:

In [ ]:
resp = requests.get(
    'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=1920-01-01&endtime=1925-01-01&minlatitude=-56.72500008&minlongitude=-80&maxlatitude=-17.49839982&maxlongitude=-66.07534742')

Notamos que dicho objeto posee tres métodos de interés: 
- `__enter__` y `__exit__` cuya funcionalidad debería subentenderse dado lo ya visto en el curso.
- `json`: retorna el contenido codificado con json de una respuesta, si corresponde. 

Observemos el contenido `json` la respuesta obtenida.

In [ ]:
display(resp.json())
resp.close()

Podemos observar que el contenido está anidado un `dict` que contiene diferentes pares de (llaves, valor) de tipo (`str`, $\textrm{obj}$) donde $\textrm{obj}$ puede ser de tipos `dict`, `list`, `str`, `int`, `float`, `NoneType`, etc...

Para el ejercicio nos centraremos en el valor asociado a la llave `'features'`, que contiene una lista con elementos de tipo `dict` con la información relativa a cada observación. Ya que no es de interés que se estudie la estructura de este `json`, proporcionamos las de funciones lambda que aplicadas a los elementos de la lista mencionada, retornan los atributos de interés (especificados con los `str` como llaves del siguiente objeto `funciones_dict`).   

**OBS**: además de el acceso elementos del diccionario `json`, la función asociada al `'timestamp'` interpretar los valores recolectados como objetos `datetime` de la librería `datetime`.

In [ ]:
funciones_dict = {
    'id': lambda x: x['id'],
    'lugar': lambda x: x['properties']['place'],
    'timestamp': lambda x: datetime.fromtimestamp(
        x['properties']['time'] // 1000),
    'magnitud': lambda x: x['properties']['mag'],
    'longitud': lambda x: x['geometry']['coordinates'][0],
    'latitud': lambda x: x['geometry']['coordinates'][1]
}

Así para obtener los atributos de interés de la primera observación aplicamos:

In [ ]:
 {k: func(resp.json()['features'][0]) for k, func in funciones_dict.items()}

6. Recolecte los datos haciendo uso de `funciones_dict`, en el siguiente procedimiento complete las 3 lineas indicadas con:
    ```python
    _ _ _ _ # Completar {n}
    ```
    - **Completar 1**: haga uso de la función `map`
    - **Completar 2** y **Completar 3**: haga lo que corresponda para obtener un `DictDeListas` con la concatenación de todos los atributos de interés para todas las consultas realizadas a la API. _Hint_ : recuerde la razón por la que definimos `DictDeListas`.

In [ ]:
# inicializar el contenedor de los datos recolectados
observaciones_dict = DictDeListas()

# generar las url de las consultas a la API
for consulta_str in GeneradorConsultaUSGS(5):

    with requests.get(consulta_str) as respuesta:

        assert respuesta.ok, f'Respuesta erronea de codigo: {respuesta.status_code}'

        respuesta_dict = respuesta.json()
        respuesta_features_lista = respuesta_dict['features']

        # reporta numero de resultados
        print(
            f'{len(respuesta_dict["features"])} observaciones en consulta {consulta_str}')

        iter_dict = DictDeListas()

        for k, func in funciones_dict.items():

            iter_dict[k] = list(map(func, respuesta_features_lista))

        if observaciones_dict:

            observaciones_dict.extiende_listas(iter_dict)

        else:

            observaciones_dict = DictDeListas(iter_dict.copy())

        # espera 5 segundos antes de realizar la próxima consulta
        sleep(5)

- Compruebe que la cantidad de observaciones recolectadas debería ser mayor a  63.500

In [ ]:
print(f'Fueron recolectadas {len(observaciones_dict["id"]):,} observaciones')

## Análisis de datos
Finalmente es posible proceder al análisis de los datos. Para ello se hará uso de  las librerías `numpy` y `scipy`. 

El primer paso será transformar las listas en numpy array segun usamdo los siguientes tipos de datos:

In [ ]:
dtype_dict = {
    'id': np.dtype('<U64'),
    'lugar': np.dtype('<U64'),
    'timestamp': np.dtype('<M8[ns]'),
    'magnitud': np.dtype(float),
    'longitud': np.dtype(float),
    'latitud': np.dtype(float)
}

7. Transforme las listas de los atributos de interés contenidas en `observaciones_dict` en un `numpy.ndarray` de `dtype` específico.  Para saber que dtype corresponde a cada atributo, guíese por `dtype_dict`. Es decir, por ejemplo, para los atributos contenidos en `observaciones_dict['timestamp']` usted debe transformarlos en un `numpy.ndarray` de `dtype` `np.dtype('<M8[ns]')` 
    - Utilice comprensión de diccionarios y el método `items`.

In [ ]:
observaciones_arr_dict = {k: np.array(v, dtype=dtype_dict[k]) for k, v in observaciones_dict.items()}

- Compruebe que el evento más antiguo contenido en `observaciones_arr_dict` corresponde al 3 de agosto de 1920.

In [ ]:
observaciones_arr_dict['timestamp'].min()

Como fue enunciado al comienzo del documento, el objetivo del ejercicio es obtener estadísticas descriptivas de los movimientos telúricos ocurridos en Chile en los últimos 100 años. Para mostrar dichas estadísticas definiremos la función `muestra_estadisticas_descriptivas` que toma como argumento un  `numpy.ndarray` y muestra en pantalla las siguientes estadísticas según la clase de los elementos contenidos en dicho arreglo:
- `'np.float64'` y `numpy.datetime64` (`datetime` en nanosegundos): número de observaciones, promedio, mediana, primer cuartil, tercer cuartil, minimo y maximo.
- `'<U64'` (`str` de largo máximo 64): número de observaciones y moda.

Como el comportamiento debe variar según el tipo de dato del argumento, sería conveniente usar el decorador `singledispatch`. Sin embargo al verificar:
```python
>>>[val.__class__ for val in observaciones_arr_dict.values()]
[numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray]
```
observamos que el todos los array contenidos en `observaciones_arr_dict` pertenecen a la misma clase, `numpy.ndarray`. Por lo tanto es necesario volver a modificar el comportamiento de `singledispatch` para que sea capaz de distinguir entre el `dtype` del array que recibe como argumento.

8. Complete el código de la siguiente función para extender la funcionalidad de `singledispatch` para distinguir `dtype` del `numpy.ndarray` usado como argumento.  
_Hint_ : inspirese de el código de `singledispatchmeth` en la Sección Preliminares.

In [ ]:
def numpydispatch(func):
    
    despachador = singledispatch(func)

    def wrapper(*args, **kw):
        return despachador.dispatch(args[0][0].__class__)(*args, **kw)
    
    wrapper.register = despachador.register
    wrapper.registry = despachador.registry
    update_wrapper(wrapper, func)
    return wrapper

- Compruebe que el resultado es el deseado:

In [ ]:
@numpydispatch
def test_numpydispatch(array):
    pass
    
@test_numpydispatch.register(np.float64)
def _(array):
    pass
        
@test_numpydispatch.register(np.datetime64)
def _(array):
    pass
    
@test_numpydispatch.register(str)
def _(array):
    pass

test_numpydispatch.registry.keys()

9. Para concluir complete las funciones de `muestra_estadisticas_descriptivas` asociadas a arreglos que contienen elementos de clase `np.float64` y `np.str_`. Estas deben mostrar en pantalla:
    - `'np.float64'`: número de observaciones, promedio, mediana, primer cuartil, tercer cuartil, minimo y maximo.
    - `'<U64'` (`str` de largo máximo 64): número de observaciones y moda.  
    
   **OBS**: se provee la función asociada a los array con elementos de la clase `np.datetime64` pues el método `view` está fuera del alcance del ejercicio.

In [ ]:
@numpydispatch
def muestra_estadisticas_descriptivas(array):
    '''Define el comportamiento para objetos en general fuera de registry'''
    print(f"numero de observaciones: {len(array):,}")
        
@muestra_estadisticas_descriptivas.register(np.datetime64)
def _(array):
    '''Define el comportamiento para los objetos '''
    
    # transforma array para que puedan calcularse las estadisticas deseadas
    trans_array = observaciones_arr_dict['timestamp'].view('i8')
    
    # inicializa el contenedor de los str a mostrar en pantalla
    lista_str = []
    
    # genera los str a mostrar en pantalla
    lista_str.append(f"numero de observaciones: {len(array):,}")
    lista_str.append(
        f"media: {trans_array.mean().astype('datetime64[ns]')}"
    )
    lista_str.append(
        f"mediana: {np.median(trans_array).astype('datetime64[ns]')}"
    )
    lista_str.append(
        f"primer cuartil: {np.quantile(trans_array, .25).astype('datetime64[ns]')}"
    )
    lista_str.append(
        f"tercer cuartil: {np.quantile(trans_array, .75).astype('datetime64[ns]')}"
    )
    lista_str.append(
        f"minimo: {trans_array.min().astype('datetime64[ns]')}"
    )
    lista_str.append(
        f"maximo: {trans_array.max().astype('datetime64[ns]')}"
    )
    
    # muestra los elementos de la lista separados por un salto de linea
    print('\n'.join(lista_str))
    
@muestra_estadisticas_descriptivas.register(np.float64)
def _(array):
    lista_str = []
    lista_str.append(f"numero de observaciones: {len(array):,}")
    lista_str.append(f"media: {np.nanmean(array):.2f}")
    lista_str.append(f"mediana: {np.nanmedian(array):.2f}")
    lista_str.append(f"primer cuartil: {np.nanquantile(array, .25):.2f}")
    lista_str.append(f"tercer cuartil: {np.nanquantile(array, .75):.2f}")
    lista_str.append(f"minimo: {np.nanmin(array)}")
    lista_str.append(f"maximo: {np.nanmax(array)}")
    print('\n'.join(lista_str))
    
    
@muestra_estadisticas_descriptivas.register(np.str_)
def _(array):
    
    array
    resultados_moda = stats.mode(array, nan_policy="omit")
    
    lista_str = []
    lista_str.append(f"numero de observaciones: {len(array):,}")
    lista_str.append(f'moda: {resultados_moda.mode[0]} ({resultados_moda.count[0]:,} ocurrencias)')
    print('\n'.join(lista_str))

- Comprobamos las siguientes estadísticas descriptivas:
    - moda('lugar'): San Juan, Argentina (más de 8.000 ocurrencias)
    - mínimo('timestamp'): 1920-08-03
    - media('magnitud'): aprox. 3.8
    - mediana('longitud'): aprox. -70.7
    - maximo('latitud'): aprox. -17.5

In [ ]:
for k, val in observaciones_arr_dict.items():
    
    if k != 'id': 
    
        print(f'\nVariable: {k}')
        muestra_estadisticas_descriptivas(val)